# Joke Scorer Notebook
## Ryan Jenkinson, Angus Brayne, Éanna Morley (Group 2)

This notebook will be a front-end that accompanies our final NLP Group Project for COMP0087 Statistical Natural Language Processing. This is intended to be an interactive experience rather than containing full implementations, model choices/architecture and hyperparameter searches. These can be found in notebook **JokeScorer_SourceCode** under the SOURCE zip file. We hope you enjoy it!

## Instructions for using the notebook
We have tested running this code in Colab, The instructions for running that are as follows:

> If you prefer to run this in Colab, then download the PUBLIC folder from Google Drive. The link is [here](https://drive.google.com/drive/folders/1jZoV809eVOAq8ZdlFkF6DTiBhLlkclkO?usp=sharing). Then, in Google Drive, click on the folder name and select Add to Drive. Then, the following code with work (using Google Drive Authentication).  We recommend that you run this on a GPU.

## Import dependencies
Including loading any relevant packages and tools that we need

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive/')

OUTPUT_BASE_DIR = "/content/gdrive/My Drive/PUBLIC/"
GLOVE_PATH = OUTPUT_BASE_DIR+'glove.6B.300d.txt'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
# Clone github repo for this project - takes some time
#!git clone https://github.com/RyanJenkinson/nlp-jcag.git

In [0]:
#@title IMPORT DEPENDENCIES

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import pickle
import re
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
import plotly.plotly as py
import plotly.graph_objs as go

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot, Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, Lambda, Bidirectional, LSTM 
from keras.layers import Dropout, concatenate, GlobalMaxPooling1D, Activation 
from keras.layers import MaxPooling1D, Input, Conv1D
from keras.layers.normalization import BatchNormalization
from keras.backend import mean
from keras.utils import to_categorical, plot_model
from keras import optimizers, initializers, regularizers
from keras import backend as K

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Contours, Histogram2dContour, Marker, Scatter

from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

!pip install bert-tensorflow

import tensorflow_hub as hub
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
from bert import run_classifier
import bert

nltk.download('stopwords')
nltk.download('wordnet')

# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

Using TensorFlow backend.


W0317 12:17:52.532921 140439917234048 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#@title WORKER FUNCTIONS
# Define some key metrics
def precision(y_true, y_pred):	
    """Precision metric.	
    Only computes a batch-wise average of precision. Computes the precision, a
    metric for multi-label classification of how many selected items are
    relevant.
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))	
    precision = true_positives / (predicted_positives + K.epsilon())	
    return precision

def recall(y_true, y_pred):	
    """Recall metric.	
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """	
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))	
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))	
    recall = true_positives / (possible_positives + K.epsilon())	
    return recall

def f1_score(y_true, y_pred):
    """Computes the F1 Score
    Only computes a batch-wise average of recall. Computes the recall, a metric
    for multi-label classification of how many relevant items are selected.	
    """
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return (2 * p * r) / (p + r + K.epsilon())

# Define some helper functions that we use to process the data
def preprocess(words, to_lowercase=True, remove_punctuation=False, remove_digits=False, 
               remove_odd_chars=True, remove_stopwords=False, stem=False, spell_check=False,
               lemmatize=False):
    
    if to_lowercase:
        words = [w.lower() for w in words]
    
    if remove_punctuation:
        words = [w for w in words if not (re.match(r'^\W+$', w) != None)]
    
    if remove_digits:
        words = [w for w in words if not w.replace('.','',1).isdigit()]

    if remove_odd_chars:
        words = [re.sub(r'[^a-zA-Z0-9_]','', w) for w in words]
    
    if remove_stopwords:
        sw = set(nltk.corpus.stopwords.words("english"))
        words = [w for w in words if not w in sw]

    if spell_check:
        words = [str(TextBlob(w).correct()) for w in words]
     
    if stem:
        porter = nltk.PorterStemmer()
        words = [porter.stem(w) for w in words]

    if lemmatize:
        wordnet = nltk.WordNetLemmatizer()
        words = [wordnet.lemmatize(w) for w in words]
        
    return words

def lookup_index(docs):
    output_matrix = []
    for d in docs:
        indices = []
        for w in d:
            if w in word2index.keys():
                indices.append(word2index[w])
            else:
                indices.append(word2index['unk'])
        output_matrix.append(indices)
    return output_matrix
  
# Define a helper function to encode the jokes via GloVe
def encode_jokes(df):
  # Concatonate set-up and punchline
  docs = [str(df.title[i]) + ' ' + str(df.body[i]) for i in range(df.shape[0])]

  # Pre-process lowercasing, removing odd characters etc.
  docs_cleaned = [preprocess(words.split()) for words in docs]

  # Encode words using GloVe
  docs_encoded = lookup_index(docs_cleaned)
  
  # Pad sentences to max length, in our clean dataset this is 70
  docs_padded = pad_sequences(docs_encoded, maxlen=70, padding='post')
  
  return docs_padded

def encode_singular_joke(jokestring):
  docs = [str(jokestring)]

  # Pre-process lowercasing, removing odd characters etc.
  docs_cleaned = [preprocess(words.split()) for words in docs]

  # Encode words using GloVe
  docs_encoded = lookup_index(docs_cleaned)
  
  # Pad sentences to max length, in our clean dataset this is 70
  docs_padded = pad_sequences(docs_encoded, maxlen=70, padding='post')
  
  return docs_padded

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        return {"eval_accuracy": accuracy}

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def joke_rater(joke_in):
  
  # We'll set sequences to be at most 128 tokens long.
  #tf.logging.set_verbosity(False)
  MAX_SEQ_LENGTH = 70
  label_list=[0,1,2]

  OUTPUT_DIR = OUTPUT_BASE_DIR
  joke_in = pd.DataFrame({'joke': ["not a joke",joke_in], 'class': [0, 1]})
  # Use the InputExample class from BERT's run_classifier code to create examples from the data
  test_InputExamples = joke_in.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                     text_a = x['joke'], 
                                                                     text_b = None, 
                                                                     label = x['class']), axis = 1)

  tokenizer = create_tokenizer_from_hub_module()

  # Convert our train and test features to InputFeatures that BERT understands.
  test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

  run_config = tf.estimator.RunConfig(
      model_dir=OUTPUT_DIR,
      save_summary_steps=0,
      save_checkpoints_steps=0)

  model_fn = model_fn_builder(
    num_labels=len(label_list),
    learning_rate=0,
    num_train_steps=0,
    num_warmup_steps=0)

  estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": 32})

  test_input_fn = run_classifier.input_fn_builder(
      features=test_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=False)

  predictions = pd.DataFrame(estimator.predict(input_fn=test_input_fn, checkpoint_path=OUTPUT_DIR+"model.ckpt-3770"))
  predicted_label = predictions['labels'].iloc[-1]

  print(" ")
  print(" ")
  if predicted_label == 0:
    print("Your joke scored %i!  https://www.scienceofpeople.com/how-to-be-funny/" %(predicted_label+1))
  elif predicted_label == 1:
    print("Your joke scored %i!  https://giphy.com/search/little-chuckle" %(predicted_label+1))
  else:
    print("Your joke scored %i!  https://tenor.com/search/laughing-hard-gifs" %(predicted_label+1))

In [0]:
#@title Try our BERT joke rater on some of your material!  The rating will print below.
joke_in = "I think it’s wrong that only one company makes the game Monopoly." #@param
joke_rater(joke_in)

Instructions for updating:
Colocations handled automatically by placer.


W0317 12:17:54.378012 140439917234048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0317 12:18:02.272961 140439917234048 deprecation.py:506] From <ipython-input-4-0c99206a35c6>:154: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0317 12:18:03.254879 140439917234048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


 
 
Your joke scored 3!  https://tenor.com/search/laughing-hard-gifs


### t-SNE Visualization

In [0]:
#@title 1. Setup

MAX_SEQUENCE_LENGTH = 70
MAX_NUM_WORDS = float('inf')
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

df = pd.read_csv(OUTPUT_BASE_DIR+'Kim_df_test.csv')

texts = [str(df.title[i]) + ' ' + str(df.body[i]) for i in range(df.shape[0])]
texts = [preprocess(words.split()) for words in texts]

embeddings_index = {}
with open(GLOVE_PATH, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
        tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

labels = df['class']

num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
K.clear_session()
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1], 
                            input_length=70,
                            weights=[embedding_matrix], 
                            trainable=False,
                            mask_zero=True,
                            name='embedding_layer')

model = Sequential()
model.add(embedding_layer)
model.add(Lambda(lambda x: K.mean(x, axis=1), name='mean_embedding'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

get_mean_embedding = K.function([model.layers[0].input],
                                [model.layers[1].output])
mean_embeddings = get_mean_embedding([data])[0]

embeddings_norm = normalize(embedding_matrix, axis=1, norm='l2')
mean_embeddings_norm = normalize(mean_embeddings, axis=1, norm='l2')

most_similar_idx = np.argmax(mean_embeddings_norm @ embeddings_norm.T, axis=1)
most_similar_wrd = [list(word_index.keys())[i] for i in most_similar_idx]

In [0]:
#@title 2. Run t-SNE

seed = 123 #@param 
perplexity = 30 #@param
tsne = TSNE(n_components=2, 
            random_state=seed, 
            verbose=0, 
            perplexity=perplexity)
transformed_weights = tsne.fit_transform(mean_embeddings)

In [0]:
#@title 3. Explore!
configure_plotly_browser_state()
init_notebook_mode(connected=False)

x = transformed_weights[:,0]
y = transformed_weights[:,1]

trace = go.Scatter(
    x=x,
    y=y,
    mode='markers',
    text=['Class: ' + str(df['class'][i]) + '<br>' + 
          'Mean word: ' + str(most_similar_wrd[i]) + '<br>' + 
          str(df.title[i]) + '<br>' + 
          str(df.body[i]) for i in range(len(most_similar_wrd))],
    marker=dict(
        color=[len(joke) for joke in texts],
        size=6,
        line=dict(
            color=[len(joke) for joke in texts],
            width=0.5,
            colorscale='Viridis'
        ),
        colorscale='Viridis',
        opacity=0.8
    ),
)

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Output hidden; open in https://colab.research.google.com to view.

The above plot is coloured by joke length. 